In [651]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
import time
import urllib
from tqdm import tqdm
import sys
import os

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import re
import datetime
import json
from IPython.display import display

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from lxml.html import fromstring
from bs4 import BeautifulSoup

In [3]:
# 수집 날짜 지정(전일자)
before_one_day = datetime.datetime.now() - datetime.timedelta(days=1)
D, db_date = before_one_day.strftime("%Y.%m.%d"), before_one_day.strftime("%y%m%d")

In [153]:
conn = db_connection('')
news_text = pd.read_sql(f"select * from db_name;", conn)
conn.close()

In [154]:
print(len(news_text))
news_text.head()

9455


,id,news,title,href,text,write_date,regist_date
0,1,TV리포트,"20년 미스터리 풀렸다.. 새벽2시 日서 사라진 김국진 ""강수지 만났다""('갓경규')",https://entertain.naver.com/now/read?oid=213&a...,"20년 미스터리가 풀린 순간, 이경규는 전율하고 김국진은 웃었다. 무슨 사연일까.1...",2023-11-01,2023-11-02 06:48:38
1,2,일간스포츠,[TVis] 딘딘 “’세상에 이런 일이’ MC 예상 못해…’포스트 임성훈’ 고려한 ...,https://entertain.naver.com/now/read?oid=241&a...,사진제공=MBC 캡처 가수 딘딘이 ‘세상에 이런 일이’ MC를 맡은 소감을 밝혔다....,2023-11-01,2023-11-02 06:48:38
2,3,스포츠조선,"""계속 토익강사하지"" 17기 옥순, 영철과 대화 중 눈물 ""연애 기회 없어"" ('나...",https://entertain.naver.com/now/read?oid=076&a...,[스포츠조선닷컴 이우주 기자] 나는 솔로 옥순이 복합적인 자신의 상황에 결국 눈물을...,2023-11-01,2023-11-02 06:48:38
3,4,OSEN,'고딩엄빠4' 극단적 선택 시도→극적 화해···엄마+딸 18세 임신 사연[종합],https://entertain.naver.com/now/read?oid=109&a...,[OSEN=오세진 기자] MBN ‘어른들은 모르는 고딩엄빠4’(이하 ‘고딩엄빠4’)...,2023-11-01,2023-11-02 06:48:38
4,5,마이데일리,"열연이 섬세하다…전종서, '발레리나'로 도약했다",https://entertain.naver.com/now/read?oid=117&a...,배우 전종서 / 넷플릭스 배우 전종서 / 넷플릭스 배우 전종서 / 넷플릭스[마이데일...,2023-11-01,2023-11-02 06:48:38


---

In [644]:
email_1 = r"[a-zA-Z0-9._+-]+@[a-zA-Z0-9]+\.[a-zA-Z]{2,4}.[a-zA-Z]{2,4}"
email_2 = r"[a-zA-Z0-9._+-]+@[a-zA-Z0-9]+\.[a-zA-Z]{2,4}"
pattern_1 = r'\[[^.]*기자\]'
pattern_2 = r'\([^)]*기자\)'
pattern_3 = r'\([^)]*뉴스\)'
pattern_4 = r'\([^)]*뉴스1\)'
pattern_5 = r'\([^)]*캡처\)'
pattern_6 = r'\[[^)]*캡처\]'
pattern_7 = r'\<[^)]*기자\>'
pattern_8 = r'ⓒ\D*(\s*)기자'
pattern_9 = r'\[[^)]*에디터\]'
pattern_10 = r'\[[^.]*뉴시스\]'
pattern_11 = r'\[[^)]*리뷰\]'
pattern_12 = r'/(\s*)사진제공(\W*)(\w*)(\d*)([^a-zA-Z가-힣]*) '
pattern_13 = r'/(\s*)사진(\W*)(\w*)(\d*)([^a-zA-Z가-힣]*) '
pattern_14 = r'(\w*) 기자 ='

In [645]:
news_text['text'] = [news_text['text'][i].replace('이미지 원본보기', '').strip() for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_1, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('[서울=뉴시스]', '') for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_2, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_3, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_4, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_5, '', news_text['text'].loc[i], 1) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_6, '', news_text['text'].loc[i], 1) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_7, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_8, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_9, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_10, '', news_text['text'].loc[i], 1) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_11, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_12, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_13, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(pattern_14, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(email_1, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [re.sub(email_2, '', news_text['text'].loc[i]) for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('\xa0', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('/사진=', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('사진=', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('./', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('- ', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('[사진]', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('[뉴스투데이]', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].replace('*재판매 및 DB 금지', '') for i in range(len(news_text))]
news_text['text'] = [news_text['text'].loc[i].strip() for i in range(len(news_text))]

In [649]:
news_text

,id,news,title,href,text,write_date,regist_date
0,1,TV리포트,"20년 미스터리 풀렸다.. 새벽2시 日서 사라진 김국진 ""강수지 만났다""('갓경규')",https://entertain.naver.com/now/read?oid=213&a...,"20년 미스터리가 풀린 순간, 이경규는 전율하고 김국진은 웃었다. 무슨 사연일까.1...",2023-11-01,2023-11-02 06:48:38
1,2,일간스포츠,[TVis] 딘딘 “’세상에 이런 일이’ MC 예상 못해…’포스트 임성훈’ 고려한 ...,https://entertain.naver.com/now/read?oid=241&a...,사진제공=MBC 캡처가수 딘딘이 ‘세상에 이런 일이’ MC를 맡은 소감을 밝혔다.딘...,2023-11-01,2023-11-02 06:48:38
2,3,스포츠조선,"""계속 토익강사하지"" 17기 옥순, 영철과 대화 중 눈물 ""연애 기회 없어"" ('나...",https://entertain.naver.com/now/read?oid=076&a...,나는 솔로 옥순이 복합적인 자신의 상황에 결국 눈물을 보였다. 1일 방송된 SBS...,2023-11-01,2023-11-02 06:48:38
3,4,OSEN,'고딩엄빠4' 극단적 선택 시도→극적 화해···엄마+딸 18세 임신 사연[종합],https://entertain.naver.com/now/read?oid=109&a...,MBN ‘어른들은 모르는 고딩엄빠4’(이하 ‘고딩엄빠4’) 고딩엄마 최고은이 결혼 ...,2023-11-01,2023-11-02 06:48:38
4,5,마이데일리,"열연이 섬세하다…전종서, '발레리나'로 도약했다",https://entertain.naver.com/now/read?oid=117&a...,배우 전종서 / 넷플릭스 배우 전종서 / 넷플릭스 배우 전종서 / 넷플릭스 배우 전...,2023-11-01,2023-11-02 06:48:38
...,...,...,...,...,...,...,...
9450,9452,TV리포트,"박은빈 ""실제 무인도에서 '무인도의 디바' 촬영... 나혼자라 더 힘들어""",https://entertain.naver.com/now/read?oid=213&a...,배우 박은빈이 실제 무인도에서 ‘무인도의 디바’를 촬영했다며 남모를 고충을 전했다....,2023-11-02,2023-11-02 23:47:53
9451,9453,뉴시스,"'51세 미혼' 장서희 ""속옷가게 마네킹도 짝 있더라""…외로움 토로",https://entertain.naver.com/now/read?oid=003&a...,배우 장서희 2023.11.01 *재판매 및 DB 배우 장서희(51)가 결혼에...,2023-11-02,2023-11-02 23:47:53
9452,9454,OSEN,"브브걸 유정, '11살 차이' ♥이규한과 열애 스토리 공개! ""고백 기다렸다"" ('...",https://entertain.naver.com/now/read?oid=109&a...,유정이 이규한과의 열애 소식 비하인드를 전했다.1일 방송된 MBC 라디오스타에서는 ...,2023-11-02,2023-11-02 23:47:53
9453,9455,엑스포츠뉴스,"유정 ""♥이규한과 열애설 후 진짜 연애...축하해 달라""(라스)[종합]",https://entertain.naver.com/now/read?oid=311&a...,라디오스타에서 유정이 현재 열애 중인 이규한을 언급했다.1일 방송된 MBC 예능 프...,2023-11-02,2023-11-02 23:47:53


In [ ]:
news_text.to_excel(f'news_text_{db_date}.xlsx', index=False)

In [650]:
user_nm = db_id
passwd = db_pw
host_url = host_url
port_num = port_num
db_name = db_name
engine = create_engine(f'')
engine_conn = engine.connect()

# insert
news_text_copy.to_sql(f'table_name', con=engine, if_exists='append', index=None)
engine_conn.close()
engine.dispose()